Himani Parikh - 1322085

CSCI 636 - Big Data Analytics

Hands on 10.2 : Spark-Classification using decision decision tree(low-humidity days vs. non-low- humidity days at 9am)


***Install pyspark***

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=e6e88c870fb7681ef2ccdfaafa9824f3fab7365482178ce00a78a5fa1f0545e2
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [29]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

spark = SparkSession\
        .builder\
        .master('local[*]')\
        .getOrCreate()
sc = spark.sparkContext

***Mount Google Drive***

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
# change the directory to big-data-colab folder
%cd /content/drive/My Drive/Colab Notebooks/BigDataColabHimaniParikh

#pwd
!ls

/content/drive/My Drive/Colab Notebooks/BigDataColabHimaniParikh
 10.Spark-ClassificationHimani.ipynb	        out
 2.DataCopyHimani.ipynb			        out_Datajoin
 2_WordCountHimani.ipynb		        outDatajoin2
 3.1_WordGrepHimani.ipynb		        out_grep
 3.2_WordMedianHimani.ipynb		        out_testfile
 4.MR-DataJoinHimani.ipynb		        out_testfile1
 4.MR-WordCountHimani.ipynb		        out_testfile2
 5.MR-DataJoin2Himani.ipynb		        out_wordmedian
 6.MR-WordCountReducerHimani.ipynb	        Prediction
 7.Spark-WordCountHimani.ipynb		        SparkOutput
 8.Spark-SQLHimani.ipynb		        SqlDataSet
 8.Spark-StreamingHimani.ipynb		        testfile1
 9.Spark-Handling-missing-values-Himani.ipynb   testfile2
 BigDataHimani.ipynb			        WeatherDataset
 Data					        wordcount_mapper.py
 Data_join				        wordcount_num0_output.txt
 Data_join2				        wordcount_num2_output.txt
'Hands On 9.2-Himani.ipynb'		        wordcount_reducer.py


***Load classes and data***

In [32]:
from pyspark.sql import SQLContext
from pyspark.sql import DataFrameNaFunctions

from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import Binarizer
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer

In [34]:
sqlsc = SQLContext(sc)

df = sqlsc.read.load('/content/drive/My Drive/Colab Notebooks/BigDataColabHimaniParikh/WeatherDataset/daily_weather.csv',
                     format='com.databricks.spark.csv',
                     header='true',
                     inferSchema='true')


***Question 1.***


---


Spark: What values are in the number column?

A. Integer values starting at 0

B. Time and date values

C. Random integer values

D. None of the above

***Answer: A***
✅

In [35]:
df.show(10)

+------+-----------------+------------------+----------------------+------------------+----------------------+------------------+---------------------+-----------------+---------------------+---------------------+
|number| air_pressure_9am|      air_temp_9am|avg_wind_direction_9am|avg_wind_speed_9am|max_wind_direction_9am|max_wind_speed_9am|rain_accumulation_9am|rain_duration_9am|relative_humidity_9am|relative_humidity_3pm|
+------+-----------------+------------------+----------------------+------------------+----------------------+------------------+---------------------+-----------------+---------------------+---------------------+
|     0|918.0600000000087| 74.82200000000041|                 271.1| 2.080354199999768|    295.39999999999986| 2.863283199999908|                  0.0|              0.0|    42.42000000000046|   36.160000000000494|
|     1|917.3476881177097| 71.40384263106537|    101.93517935618371|2.4430092157340217|    140.47154847112498|3.5333236016106238|               

In [36]:
df.printSchema()

root
 |-- number: integer (nullable = true)
 |-- air_pressure_9am: double (nullable = true)
 |-- air_temp_9am: double (nullable = true)
 |-- avg_wind_direction_9am: double (nullable = true)
 |-- avg_wind_speed_9am: double (nullable = true)
 |-- max_wind_direction_9am: double (nullable = true)
 |-- max_wind_speed_9am: double (nullable = true)
 |-- rain_accumulation_9am: double (nullable = true)
 |-- rain_duration_9am: double (nullable = true)
 |-- relative_humidity_9am: double (nullable = true)
 |-- relative_humidity_3pm: double (nullable = true)



In [37]:
featureColumns = ['air_pressure_9am',
 'air_temp_9am',
 'avg_wind_direction_9am',
 'avg_wind_speed_9am',
 'max_wind_direction_9am',
 'max_wind_speed_9am',
 'rain_accumulation_9am',
 'rain_duration_9am']

***Drop unused and missing data***

In [38]:
df = df.drop('number')

In [39]:
df = df.na.drop()
df.count(), len(df.columns)

(1064, 10)

***Create categorical variable***

In [40]:
binarizer = Binarizer(threshold=24.99999, inputCol='relative_humidity_3pm', outputCol='label')
binarizerDF = binarizer.transform(df)

In [41]:
binarizerDF.select('relative_humidity_3pm', 'label').show(4)

+---------------------+-----+
|relative_humidity_3pm|label|
+---------------------+-----+
|   36.160000000000494|  1.0|
|     19.4265967985621|  0.0|
|   14.460000000000045|  0.0|
|   12.742547353761848|  0.0|
+---------------------+-----+
only showing top 4 rows



***Aggregate features***

In [42]:
assembler = VectorAssembler(inputCols=featureColumns, outputCol="features")
assembled = assembler.transform(binarizerDF)

***Split training and test data 80-20***

In [43]:
(trainingData, testData) = assembled.randomSplit([0.8, 0.2], seed=13234)

In [44]:
trainingData.count(), testData.count()

(846, 218)

***Create and train decision tree***

In [45]:
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features", maxDepth=5,
                       minInstancesPerNode=20, impurity="gini")

In [46]:
pipeline = Pipeline(stages=[dt])
model = pipeline.fit(trainingData)

In [47]:
predictions = model.transform(testData)

***Question 2.***


---


Spark: With the original dataset split into 80% for training and 20% for test, how
many of the first 20 samples from the test set were correctly classified?

A. 19

B. 10

C. 1

D. None of the above

***Answer: D*** ✅(16 test set were correctly classified)

In [48]:
predictions.select("prediction", "label").show(20)

+----------+-----+
|prediction|label|
+----------+-----+
|       1.0|  1.0|
|       1.0|  1.0|
|       0.0|  1.0|
|       1.0|  1.0|
|       1.0|  1.0|
|       1.0|  1.0|
|       1.0|  1.0|
|       1.0|  1.0|
|       0.0|  0.0|
|       1.0|  1.0|
|       1.0|  1.0|
|       1.0|  1.0|
|       1.0|  1.0|
|       0.0|  0.0|
|       1.0|  1.0|
|       1.0|  1.0|
|       0.0|  1.0|
|       0.0|  1.0|
|       0.0|  1.0|
|       1.0|  1.0|
+----------+-----+
only showing top 20 rows



***Split training and test data 70-30***

***Question 3.***



---




Spark: If we split the data using 70% for training data and 30% for test data, how
many samples would the training set have (using seed 13234)?

A. 730

B. 334

C. 70

D. None of the above

***Answer: D*** ✅(As trainingData.count() = 731)


In [49]:
(trainingData, testData) = assembled.randomSplit([0.7, 0.3], seed=13234)

In [50]:
trainingData.count(), testData.count()

(731, 333)